![Banner](images/banner.png)

# Working with JSON Data

Documentation reference link: [Using JSON Data](https://python-oracledb.readthedocs.io/en/latest/user_guide/json_data_type.html)

<hr>

Setup for this notebook:

In [ ]:
import os
import oracledb

un = os.environ.get("PYO_SAMPLES_MAIN_USER", "pythondemo")
pw = os.environ.get("PYO_SAMPLES_MAIN_PASSWORD", "welcome")
cs = os.environ.get("PYO_SAMPLES_CONNECT_STRING", "localhost/orclpdb")

connection = oracledb.connect(user=un, password=pw, dsn=cs)

### JSON Storage:

- Oracle Database 12c introduced JSON stored as a LOB or VARCHAR2

- Oracle Database 21c introduced a new optimized native binary format and a dedicated JSON type

**Careful coding is required for apps that run in a mixed version environment**

The first JSON example assumes you are using Oracle Database 21c or later.

Setup the schema:

In [ ]:
with connection.cursor() as cursor:
    try:
        cursor.execute("drop table customers")
    except:
        pass
    cursor.execute("create table customers (k number, json_data json)")

With Oracle Database 21c or later, you can bind Python objects directly to the JSON column:

In [ ]:
import datetime

json_data = [
    2.78,
    True,
    'Ocean Beach',
    b'Some bytes',
    {'keyA': 1, 'KeyB': 'Melbourne'},
    datetime.date.today()
]

with connection.cursor() as cursor:
    cursor.setinputsizes(oracledb.DB_TYPE_JSON)
    cursor.execute("insert into customers (k, json_data) values (1, :jbv)", [json_data])
    
print("Done")

Querying returns the JSON in a familiar Python data structure:

In [ ]:
with connection.cursor() as cursor:
    for row, in cursor.execute("select c.json_data from customers c where k = 1"):
        print(row)
        
# With Oracle Database 21c or later, this gives:
# [Decimal('2.78'), True, 'Ocean Beach', b'Some bytes', {'keyA': Decimal('1'), 'KeyB': 'Melbourne'}, datetime.datetime(2022, 3, 4, 0, 0)]

If you don't have a recent database, then you can still easily work with JSON.  Store it using BLOB and work with JSON strings. The Python "json" package can be used with many Python types:

In [ ]:
import json

with connection.cursor() as cursor:
    try:
        cursor.execute("drop table customersblob")
    except:
        ;
    cursor.execute("""create table customersblob (k number, 
                                                  json_data blob check (json_data is json)) 
                                                        lob (json_data) store as (cache)""")
 
# INSERT

with connection.cursor() as cursor:
    data = json_data = [
        2.78,
        True,
        'Ocean Beach',
        {'keyA': 1, 'KeyB': 'Melbourne'},
    ]
    cursor.execute("insert into customersblob (k, json_data) values (2, :jbv)", [json.dumps(data)])
 
# FETCH

with connection.cursor() as cursor:
    for row, in cursor.execute("SELECT c.json_data FROM customersblob c where k = 2"):
        print(row)
    
connection.rollback()   